In [40]:
%pwd

'c:\\Users\\Asus\\OneDrive\\Desktop\\NLP_PROJECT'

In [41]:
import os
os.chdir("../")

In [42]:
%pwd

'c:\\Users\\Asus\\OneDrive\\Desktop'

In [43]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path
    

In [44]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories


In [45]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [46]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [47]:
import os
import requests
import logging
from pathlib import Path
import zipfile

class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            response = requests.get(self.config.source_URL)
            
            if response.status_code == 200:
                with open(self.config.local_data_file, 'wb') as file:
                    file.write(response.content)
                
                logging.info(f"File downloaded: {self.config.local_data_file}")
            else:
                logging.error(f"Failed to download file. Status code: {response.status_code}")
        else:
            logging.info(f"File already exists of size: {self.get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

    @staticmethod
    def get_size(file_path):
        return file_path.stat().st_size


In [50]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except zipfile.BadZipFile as zip_error:
    logging.error(f"Bad ZIP file encountered: {str(zip_error)}")
except Exception as e:
    logging.error(f"An error occurred: {str(e)}")


[2023-09-21 17:37:52,505:ERROR:An error occurred: [Errno 2] No such file or directory: 'config\\config.yaml']
